In [12]:
# HGTC_name	number	recipient_genome	recipient_scaffold	recipient_bp	donor_genome	donor_scaffold	donor_bp1	donor_bp2	reverse	recipient_MGE	recipient_MGE_category	recipient_MGE_n	donor_MGE	donor_MGE_category	donor_MGE_n	transfer_gene	transfer_gene_n	recipient_phylum	donor_phylum

# HGTC_61	10	MGYG000000045	MGYG000000045_28	6245	MGYG000004045	MGYG000004045_97	1156	3172	False	NA	NA	0	NA	NA	0	NA	0	Firmicutes	Firmicutes

import os
import pandas as pd

infile = '../../HGT_demo_file/SAMEA3449210.event_output.csv'
db_file = '../../HGT_demo_file/HGT/DB.HGT_clusters.annotated.tsv'
outdir = '.'
fr_size = 1000

df = pd.read_csv(infile, header=0, index_col=None)
db = pd.read_csv(db_file, header=0, index_col=0, sep='\t')
df.rename(columns={'receptor':'recipient'}, inplace=True)

In [13]:
def overlap(range1, range2):
    if range1[0] > range2[1] or range1[1] < range2[0]:
        return False
    else:
        return True

# styp = recipient or donor
def search_event(scaffold, db, range, stype):
    scaffold_df = db[db['{}_scaffold'.format(stype)] == scaffold]
    valid_idx = []
    if stype == 'donor':
        for idx in scaffold_df.index:
            if overlap(range, [scaffold_df.loc[idx, 'donor_bp1'], scaffold_df.loc[idx, 'donor_bp2']]):
                valid_idx.append(idx)
        valid_df = scaffold_df.loc[valid_idx, ]
    else:
        valid_df = scaffold_df[(scaffold_df['recipient_bp'] >= range[0]) & (scaffold_df['recipient_bp'] <= range[1])]
    return valid_df

def search_row(idx, df, db, fr_size):
    row = df.loc[idx, ]
    # for recipient
    recipient = row['recipient']
    recipient_range = [max(0, row['insert_locus']-fr_size), row['insert_locus']+fr_size]
    recipient_df = search_event(recipient, db, recipient_range, 'recipient')
    # for donor
    donor = row['donor']
    range = [max(0, row['delete_start'] - fr_size), row['delete_end'] + fr_size]
    donor_df = search_event(donor, db, range, 'donor')
    return recipient_df, donor_df 



In [18]:
result_anno = pd.DataFrame(columns=['id', 'sample', 'recipient_HGTC_n', 'recipient_HGTC_list', 'donor_HGTC_n', 'donor_HGTC_list', 'recipient', 'insert_locus', 'donor', 'delete_start', 'delete_end', 'reverse_flag'])
all_HGTC_idx = []
for idx in df.index:
    recipient_df, donor_df = search_row(idx, df, db, fr_size)
    all_HGTC_idx.extend(recipient_df.index)
    all_HGTC_idx.extend(donor_df.index)
    id = 'HGT_c{}'.format(idx+1)
    sample = df.loc[idx, 'sample']
    recipient_HGTC_n = recipient_df.shape[0]
    recipient_HGTC_list = ';'.join(recipient_df.index)
    if recipient_HGTC_n == 0:
        recipient_HGTC_list = 'NA'
    donor_HGTC_n = donor_df.shape[0]
    donor_HGTC_list = ';'.join(donor_df.index)
    if donor_HGTC_n == 0:
        donor_HGTC_list = 'NA'
    recipient = df.loc[idx, 'recipient']
    insert_locus = df.loc[idx, 'insert_locus']
    donor = df.loc[idx, 'donor']
    delete_start = df.loc[idx, 'delete_start']
    delete_end = df.loc[idx, 'delete_end']
    reverse_flag = df.loc[idx, 'reverse_flag']
    result_anno.loc[len(result_anno), ] = [id, sample, recipient_HGTC_n, recipient_HGTC_list, donor_HGTC_n, donor_HGTC_list, recipient, insert_locus, donor, delete_start, delete_end, reverse_flag]
    #result_anno.iloc[len(result_anno), ] = [id, sample, recipient_HGTC_n, recipient_HGTC_list, donor_HGTC_n, donor_HGTC_list, recipient, insert_locus, donor, delete_start, delete_end, reverse_flag]

db.loc[all_HGTC_idx, ].to_csv(os.path.join(outdir, 'output.HGTC_annotation.HGTC.tsv'), index=True, sep='\t')
result_anno.to_csv(os.path.join(outdir, 'output.HGTC_annotation.annotated.tsv'), index=False, sep='\t')


In [19]:
recipient_HGTC_n

1

In [21]:
result_anno

,id,sample,recipient_HGTC_n,recipient_HGTC_list,donor_HGTC_n,donor_HGTC_list,recipient,insert_locus,donor,delete_start,delete_end,reverse_flag
0,HGT_c1,SAMEA3449210,1,HGTC_2821,21,HGTC_43;HGTC_88;HGTC_101;HGTC_197;HGTC_211;HGT...,MGYG000002926_8,51904,MGYG000000070_3,80170,81829,True
1,HGT_c2,SAMEA3449210,1,HGTC_763,5,HGTC_763;HGTC_983;HGTC_1455;HGTC_1461;HGTC_3903,MGYG000001132_27,72500,MGYG000004551_1,33911,35369,True
2,HGT_c3,SAMEA3449210,1,HGTC_173,17,HGTC_173;HGTC_316;HGTC_374;HGTC_383;HGTC_429;H...,MGYG000001338_2,16000,MGYG000004481_60,119,1814,True
3,HGT_c4,SAMEA3449210,1,HGTC_73,2,HGTC_73;HGTC_354,MGYG000000003_1,491015,MGYG000001420_2,1240944,1256164,True
4,HGT_c5,SAMEA3449210,1,HGTC_2822,1,HGTC_2822,MGYG000001940_46,794,MGYG000002661_58,4674,5482,False
5,HGT_c6,SAMEA3449210,1,HGTC_2823,199,HGTC_5;HGTC_11;HGTC_13;HGTC_22;HGTC_67;HGTC_68...,MGYG000000003_6,93988,MGYG000003084_128,574,1931,False
6,HGT_c7,SAMEA3449210,1,HGTC_85,38,HGTC_85;HGTC_328;HGTC_544;HGTC_545;HGTC_567;HG...,MGYG000002143_1,89223,MGYG000002156_4,72097,80022,False
7,HGT_c8,SAMEA3449210,1,HGTC_2824,3,HGTC_2824;HGTC_3555;HGTC_3650,MGYG000002673_33,1879,MGYG000001698_6,17050,25240,True
8,HGT_c9,SAMEA3449210,1,HGTC_2825,1,HGTC_2825,MGYG000002416_12,54320,MGYG000003004_57,536,1349,True
9,HGT_c10,SAMEA3449210,1,HGTC_2826,4,HGTC_2697;HGTC_2826;HGTC_3274;HGTC_3275,MGYG000002454_1,1391624,MGYG000001921_7,61159,91866,True
